<a href="https://colab.research.google.com/github/gmazzitelli/be-a-citizen-scientist/blob/master/BeACitizenScientist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GET data from URL
Carica il file del questionario dalla URL (shared readeable file)
buona refence [link text](https://michaelsaruggia.com/python-per-data-science-pandas/)

In [0]:
import numpy as np
import pandas as pd
url_csv_file = "https://docs.google.com/spreadsheet/ccc?key=19zCRezCS3yIy6_Igzex2sotnNja2Ks57RonGX8W2R8I&output=csv"
df = pd.read_csv(url_csv_file).replace(np.nan, 'Nessuna risposta', regex=True)


qualche aggiustamento dei dati per distinguere grandi da piccoli e togliere sotituire un domanda con la virgala che la fa comparire come multiple choiche

In [0]:
df = df.replace({'Età in anni compiuti', '75 e oltre'}, {'Età in anni compiuti', 'tra 75 e oltre'} )
from sklearn.preprocessing import OrdinalEncoder
df['young'] = np.where(df['Età in anni compiuti']<'tra 25 e 34', 1, 0)
df['Sai quali pericoli potrebbero esserci nel postare le foto sui social?']=df['Sai quali pericoli potrebbero esserci nel postare le foto sui social?'].apply(
    lambda x: 'lo so - ma non mi interessa' if x == 'lo so, ma non mi interessa' else x)
young=df[df.young==1].copy()
old =df[df.young==0].copy()
df.head()

# dataset visualization

visualizza tutti dati in modo basico differenziando fra grandi e piccoli

In [0]:
#@title Default title text
def multiple_choice(question, db):
  multiple_answar = db[question].str.split(',\s*').apply(lambda x: pd.Series(1,index=x)).fillna(0)
  labels=[]
  values=[]
  for i in range (0, len(multiple_answar.columns)):
    labels.append(multiple_answar.columns[i])
    values.append(100*multiple_answar[multiple_answar.columns[i]].sum())
  return values, labels

show_as_pie = 6
show_as_hist = 40

from sklearn.preprocessing import OrdinalEncoder
for i in range (1, len(df.columns)):
  print (df[df.columns[i]].value_counts())
  if np.max(df[df.columns[i]].astype(str).str.find(','))>-1:
    plt.figure(figsize=(15,5))
    values,labels = multiple_choice(question=df.columns[i], db=df)
    values_y,labels_y = multiple_choice(question=df.columns[i], db=young)

    all_lables = labels + labels_y
    all_lables = list(set(all_lables))
    all_values=np.zeros(len(all_lables))
    all_values_y=np.zeros(len(all_lables))
    for i in range(0, len(all_lables)):
      if all_lables[i] in labels:
        all_values[i] = values[labels.index(all_lables[i])]
      if all_lables[i] in labels_y:
        all_values_y[i] = values_y[labels_y.index(all_lables[i])]
    
    x = np.linspace(0,len(all_lables),len(all_lables))
    plt.bar(x, np.array(all_values), label='all')
    plt.bar(x, np.array(all_values_y), color='red', label='young')
    plt.xticks(x, all_lables, rotation='vertical')
    #plt.ylabel('percentage [%]')
    plt.legend()
  else:
    ordinal_encoder = OrdinalEncoder()
    ordinated_encoded = ordinal_encoder.fit_transform(df[[df.columns[i]]])
    ordinated_encoded_young = ordinated_encoded[df.young==1]
    ordinated_encoded_old = ordinated_encoded[df.young==0]
    labels = np.array(ordinal_encoder.categories_[0])
    if len(ordinal_encoder.categories_[0])<show_as_pie :
      fig, ax = plt.subplots(1,3, figsize=(15, 5))
      val, _ = np.histogram(ordinated_encoded, bins=len(labels))
      ax[0].pie(val, labels=labels, autopct='%1.1f%%')
      ax[0].set_title('all')
      val, _ = np.histogram(ordinated_encoded_young, bins=len(labels))
      ax[1].pie(val, labels=labels, autopct='%1.1f%%')
      ax[1].set_title('young')
      val, _ = np.histogram(ordinated_encoded_old, bins=len(labels))
      ax[2].pie(val, labels=labels, autopct='%1.1f%%')
      ax[2].set_title('old')
    else:
      cat_num = np.linspace(1,len(ordinal_encoder.categories_[0]),len(ordinal_encoder.categories_[0]))
      if len(cat_num) > show_as_hist:
        cat_num = cat_num[::len(cat_num)//show_as_hist]
        labels = labels[::len(cat_num)//show_as_hist]
      plt.figure(figsize=(15,5))
      plt.hist(ordinated_encoded, range=(0,len(ordinal_encoder.categories_[0])), 
               bins=len(ordinal_encoder.categories_[0]), label='all')
      plt.hist(ordinated_encoded_young, range=(0,len(ordinal_encoder.categories_[0])), 
               bins=len(ordinal_encoder.categories_[0]), color='red', label='young')
      plt.xticks(cat_num-0.5,labels, rotation='vertical')
      plt.legend()
  plt.show()

# test area


In [184]:
def multiple_choice(question, db):
  multiple_answar = db[question].str.split(',\s*').apply(lambda x: pd.Series(1,index=x)).fillna(0)
  labels=[]
  values=[]
  print (len(multiple_answar.columns))
  for i in range (0, len(multiple_answar.columns)):
    labels.append(multiple_answar.columns[i])
    values.append(100*multiple_answar[multiple_answar.columns[i]].sum())
  return values, labels

q = 'Sai quali pericoli potrebbero esserci nel postare le foto sui social?'
#q = 'Sul web, dove prendi le informazioni? [motori di ricerca (es google, safari, ping]'
q ='Negli ultimi 12 mesi che strumenti hai utilizzato per cercare informazioni?  [Carta (es. giornali, riviste, ecc.)]'
values,labels = multiple_choice(question=q, db=df)
values_y,labels_y = multiple_choice(question=q, db=young)
print (labels)
print ("all",values, sum(values))
print (labels_y)
print ("young:",values_y, sum(values_y))
all_lables = labels + labels_y
all_lables = list(set(all_lables))
all_values=np.zeros(len(all_lables))
all_values_y=np.zeros(len(all_lables))
for i in range(0, len(all_lables)):
  if all_lables[i] in labels:
    all_values[i] = values[labels.index(all_lables[i])]
  if all_lables[i] in labels_y:
    all_values_y[i] = values_y[labels_y.index(all_lables[i])]

print (all_lables)
print (all_values)
print (all_values_y)

    

5
5
['raramente', 'mai', 'spesso', 'sempre', 'Nessuna risposta']
all [52500.0, 26300.0, 19300.0, 2100.0, 3000.0] 103200.0
['mai', 'spesso', 'raramente', 'Nessuna risposta', 'sempre']
young: [23200.0, 9200.0, 38800.0, 2100.0, 500.0] 73800.0
['spesso', 'raramente', 'sempre', 'mai', 'Nessuna risposta']
[19300. 52500.  2100. 26300.  3000.]
[ 9200. 38800.   500. 23200.  2100.]


In [0]:
df['Sai quali pericoli potrebbero esserci nel postare le foto sui social?'].apply(
    lambda x: 'lo so - ma non mi interessa' if x == 'lo so, ma non mi interessa' else x)


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
def plot_multiple_choice(question, db, figsize):
  %matplotlib inline
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots(figsize=figsize)
  multiple_answar = db[question].str.split(',\s*').apply(lambda x: pd.Series(1,index=x)).fillna(0)
  labels=[]
  values=[]
  for i in range (0, len(multiple_answar.columns)):
    labels.append(multiple_answar.columns[i])
    values.append(100*multiple_answar[multiple_answar.columns[i]].sum()/len(db))
  x = np.linspace(0,len(labels),len(labels))
  bp = ax.bar(x, np.array(values))
  plt.xticks(x, labels, rotation='vertical')
  plt.ylabel('percentage [%]')
  return bp

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
  import matplotlib.pyplot as plt
  import os
  path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
  print("Saving figure", fig_id)
  if tight_layout:
    plt.tight_layout()
  plt.savefig(path, format=fig_extension, dpi=resolution)
show_as_pie = 6
show_as_hist = 40

from sklearn.preprocessing import OrdinalEncoder
for i in range (1, len(df.columns)):
  print (df[df.columns[i]].value_counts())
  df["multiple"]=df[df.columns[i]].astype(str).str.find(',')
  if np.max(df["multiple"])>-1:
    plot_multiple_choice(question=df.columns[i], db=df, figsize=(15,5))
  else:
    ordinal_encoder = OrdinalEncoder()
    ordinated_encoded = ordinal_encoder.fit_transform(df[[df.columns[i]]])
    labels = np.array(ordinal_encoder.categories_[0])
    if len(ordinal_encoder.categories_[0])<show_as_pie :
      val, _ = np.histogram(ordinated_encoded, bins=len(labels))
      plt.figure(figsize=(5,5))
      plt.pie(val, labels=labels)
    else:
      cat_num = np.linspace(1,len(ordinal_encoder.categories_[0]),len(ordinal_encoder.categories_[0]))
      if len(cat_num) > show_as_hist:
        cat_num = cat_num[::len(cat_num)//show_as_hist]
        labels = labels[::len(cat_num)//show_as_hist]
      plt.figure(figsize=(15,5))
      plt.hist(ordinated_encoded, range=(0,len(ordinal_encoder.categories_[0])), bins=len(ordinal_encoder.categories_[0]))
      plt.xticks(cat_num-0.5,labels, rotation='vertical')
  plt.show()

# Look into data

In [0]:
df.info()

In [0]:
df.describe()

In [0]:
print(df['Cittadinanza'].value_counts())

# info
jupyter notebook: https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

selezionare elemeni in array o liste: https://teamtreehouse.com/community/how-do-i-get-the-first-4-and-last-4-items-of-a-list

machine leaenring https://medium.com/botsupply/il-machine-learning-%C3%A8-divertente-parte-1-97d4bce99a06



# GET data from local file with autentication




In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/
!ls "/content/drive/My Drive/Colab Notebooks"
import pandas as pd
csv_file = "/content/drive/My Drive/Colab Notebooks/_Be a Data Scientist a #IJF19 (Responses) - Form Responses 1.csv"
PROJECT_ROOT_DIR = "/content/drive/My Drive/Colab Notebooks/"
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)
df = pd.read_csv(csv_file)
df.head()

# example of dataset visualizzation

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
df['Sesso'].value_counts().plot(kind='pie')
plt.show()  
df['Sesso'].value_counts()


istogramma... 
l'ordine prescelto per una visulizzazione semplice e' in base a gli entries (che falsa la visulizzazione)

In [0]:
df['Età in anni compiuti'].value_counts().plot(kind='bar')
plt.show()  
 

data are index in ASCI oder and must be reindex 

In [0]:
ordine_eta = ['tra 11 e 14', 'tra 15 e 17', 'tra 18 e 19',
       'tra 20 e 24', 'tra 25 e 34', 'tra 35 e 44', 'tra 45 e 54',
       'tra 55 e 64', 'tra 65 e 74', '75 e oltre']
df['Età in anni compiuti'].value_counts().reindex(ordine_eta).plot(kind='bar')
plt.show() 

In [0]:
cat_eta = pd.Categorical(df['Età in anni compiuti'], 
                     categories=ordine_eta, 
                     ordered=True)
cat_eta.value_counts().plot(kind='bar')
plt.show()

per passarlo ad algorirmi di machine lenrning e' necessario indicizzarlo corretamente

In [0]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[ordine_eta])
eta_encoded = ordinal_encoder.fit_transform(df[['Età in anni compiuti']])

segue esempio di unsepevised machine lerning che separa in 2 cluster
dati raw

In [0]:
print(ordinal_encoder.categories_)
plt.hist(eta_encoded)
plt.show()

applichaimo una classificazione per capire le famiglie di dati separati in 2 cluster usando KMeans

In [0]:
from sklearn import cluster
import numpy as np
X = eta_encoded
k_means = cluster.KMeans(n_clusters=2)
k_means.fit(X)
#labels = dff.columns
labels = k_means.labels_
#plt.scatter(X[:,1], X[:,0], c=labels.astype(np.float))
plt.hist(X, range=(0,len(ordinal_encoder.categories_[0])), bins=len(ordinal_encoder.categories_[0]))
plt.hist(X[labels==0], range=(0,len(ordinal_encoder.categories_[0])), bins=len(ordinal_encoder.categories_[0]),color = 'r')
plt.xticks([0,1,2,3,4,5,6,7,8,9], ordine_eta, rotation='vertical')
plt.show()
print(ordinal_encoder.categories_[0])


campi con rispote multiple: esempio con il campo "Quali argomenti ti vengono suggeriti mentre sei online"

In [0]:
question='Quali argomenti ti vengono suggeriti mentre sei online'
question='Negli ultimi 12 mesi, quando hai cercato notizie, quanto tempo hai impiegato mediamente al giorno nella ricerca?'
df["multiple"]=df[question].str.find(',')
if np.max(df["multiple"])>-1:
  print (len(df[question].str.split(',\s*')))
np.max(df["multiple"])


-1

In [0]:
multiple_answar = df['Quali argomenti ti vengono suggeriti mentre sei online'].str.split(',\s*').apply(lambda x: pd.Series(1,index=x)).fillna(0)
multiple_answar.hist(figsize=(20,15))

plt.show()

In [0]:
labels=[]
values=[]
plt.figure(figsize=(10,5))
for i in range (1, len(multiple_answar.columns)):
  #print(multiple_answar.columns[i], multiple_answar[multiple_answar.columns[i]].sum())
  labels.append(multiple_answar.columns[i])
  values.append(multiple_answar[multiple_answar.columns[i]].sum()/len(df))
x = np.linspace(0,len(labels),len(labels))
plt.bar(x, np.array(values))
plt.xticks(x, labels, rotation='vertical')
plt.ylabel('percentage')
plt.show()

In [0]:
def plot_multiple_choice(question, db, fsize):
  %matplotlib inline
  import matplotlib.pyplot as plt
  multiple_answar = db[question].str.split(',\s*').apply(lambda x: pd.Series(1,index=x)).fillna(0)
  labels=[]
  values=[]
  plt.figure(figsize=fsize)
  for i in range (1, len(multiple_answar.columns)):
    labels.append(multiple_answar.columns[i])
    values.append(multiple_answar[multiple_answar.columns[i]].sum()/len(db))
  x = np.linspace(0,len(labels),len(labels))
  plt.bar(x, np.array(values))
  plt.xticks(x, labels, rotation='vertical')
  plt.ylabel('percentage [%]')
  plt.show()

plot_multiple_choice(question='Quali social usi di più per informarti?', db=df, fsize=(10,5))
